In [1]:
# impot required packages
import requests, json
import socket  # To test internet availability

The  [ISO 4217 standard](https://currencysystem.com/codes/) defines the standard currencies with their associated three-letter codes.

In [2]:
# define our function
def convert(amount, from_curr = 'USD', to_curr = 'USD', api_key=None):
    """
    Converts currencies from source currency to target currency
    
    Args:
    -----
    amount : float
            Amount of input_curr to be converted
    from_curr : String
            Three letter code of the input currency in capital letters
    to_curr : String
            Three letter code of the output currency
    
    Returns:
    --------
    float: equivalent amount in output currency.
    """
    # Store the very first conversion rates and the corresponding date
    old_date = 'Mon, 30 Aug 2021 00:02:31 UTC'
    old_rate = {'USD': 1, 'AED': 3.67, 'AFN': 79.87, 'ALL': 103.65, 'AMD': 493.71, 'ANG': 1.79, 'AOA': 639.47,
                'ARS': 97.4, 'AUD': 1.37, 'AWG': 1.79, 'AZN': 1.7, 'BAM': 1.66, 'BBD': 2, 'BDT': 85.12, 'BGN': 1.66,
                'BHD': 0.376, 'BIF': 1975.19, 'BMD': 1, 'BND': 1.35, 'BOB': 6.89, 'BRL': 5.25, 'BSD': 1, 'BTN': 73.66,
                'BWP': 11.22, 'BYN': 2.52, 'BZD': 2, 'CAD': 1.26, 'CDF': 1988.5, 'CHF': 0.913, 'CLP': 786.66, 
                'CNY': 6.47, 'COP': 3867.5, 'CRC': 620.95, 'CUC': 1, 'CUP': 25.75, 'CVE': 93.53, 'CZK': 21.72, 
                'DJF': 177.72, 'DKK': 6.33,'DOP': 57.03, 'DZD': 135.8, 'EGP': 15.69, 'ERN': 15, 'ETB': 45.61, 
                'EUR': 0.848, 'FJD': 2.1, 'FKP': 0.727, 'FOK': 6.33, 'GBP': 0.727, 'GEL': 3.12, 'GGP': 0.727, 
                'GHS': 6.04, 'GIP': 0.727, 'GMD': 52.08, 'GNF': 9812.27, 'GTQ': 7.73, 'GYD': 209.33, 'HKD': 7.79,
                'HNL': 23.84, 'HRK': 6.39, 'HTG': 96.52, 'HUF': 298.11, 'IDR': 14456.5, 'ILS': 3.25, 'IMP': 0.727, 
                'INR': 73.66, 'IQD': 1460.62, 'IRR': 41781.6, 'ISK': 126.71, 'JMD': 152.56, 'JOD': 0.709, 'JPY': 109.93,
                'KES': 109.76, 'KGS': 84.68, 'KHR': 4083.3, 'KID': 1.37, 'KMF': 417.29, 'KRW': 1166.06, 'KWD': 0.3,
                'KYD': 0.833, 'KZT': 427.08, 'LAK': 9606.15, 'LBP': 1507.5, 'LKR': 200.03, 'LRD': 171.73, 'LSL': 14.75,
                'LYD': 4.53, 'MAD': 8.97, 'MDL': 17.66, 'MGA': 3914.08, 'MKD': 52.29, 'MMK': 1645.07, 'MNT': 2855.92,
                'MOP': 8.02, 'MRU': 36.17, 'MUR': 42.67, 'MVR': 15.34, 'MWK': 810.72, 'MXN': 20.27, 'MYR': 4.19, 
                'MZN': 63.94, 'NAD': 14.75, 'NGN': 416.6, 'NIO': 35.17, 'NOK': 8.74, 'NPR': 117.85, 'NZD': 1.43, 
                'OMR': 0.384, 'PAB': 1, 'PEN': 4.07, 'PGK': 3.51, 'PHP': 49.97, 'PKR': 166.18, 'PLN': 3.89, 'PYG': 6944.8,
                'QAR': 3.64, 'RON': 4.18, 'RSD': 100, 'RUB': 73.86, 'RWF': 1007.5, 'SAR': 3.75, 'SBD': 7.94, 'SCR': 14.52, 
                'SDG': 442.67, 'SEK': 8.66, 'SGD': 1.35, 'SHP': 0.727, 'SLL': 10332.81, 'SOS': 579.08, 'SRD': 21.45, 
                'SSP': 177.61, 'STN': 20.78, 'SYP': 1258.24, 'SZL': 14.75, 'THB': 32.72, 'TJS': 11.29, 'TMT': 3.5,
                'TND': 2.78, 'TOP': 2.24, 'TRY': 8.38, 'TTD': 6.78, 'TVD': 1.37, 'TWD': 27.89, 'TZS': 2315.97, 
                'UAH': 26.89, 'UGX': 3532.35, 'UYU': 42.78, 'UZS': 10732.83, 'VES': 4139860.26, 'VND': 22889.82, 
                'VUV': 112.42, 'WST': 2.55, 'XAF': 556.38, 'XCD': 2.7, 'XDR': 0.705, 'XOF': 556.38,
                'XPF': 101.22, 'YER': 250.63, 'ZAR': 14.75, 'ZMW': 15.98}
    
    # Check internet availability
    def is_connected():
        try:
            socket.create_connection(("1.1.1.1", 53))
            return True
        except OSError:
            pass
        return False
    
    # Base currency for rates
    base_curr = 'USD'
    
    # Define exchange url based on the availability of API key
    if api_key == None:
        url = 'https://open.er-api.com/v6/latest/'+ base_curr
    else:
        url = 'https://v6.exchangerate-api.com/v6/'+ api_key + '/latest/'+ base_curr
    
    # if connected, download new rate, else use old rate
    if is_connected():
        result = requests.get(url, timeout=15).json()
        rate_usd = result['conversion_rates']
        old_date = result['time_last_update_utc'][:-6] + ' UTC'
    else:
        print("Can't connect to internet... \nUsing old rate from {}".format(old_date))
        rate_usd = old_rate
    
    # Calculate the equivalent amount in target currency
    output_amount = (amount / rate_usd[from_curr]) * rate_usd[to_curr]

    return output_amount

Refer to [ExchangeRate-API Documentation](https://www.exchangerate-api.com/docs/overview) for more information about the **API Key**

### Testing our function

In [3]:
# When connected to internet

api = '8062c791d8d886de36c204d5'
convert(100, 'USD', 'XAF', api)

55638.329999999994

In [4]:
# Offline

api = '8062c791d8d886de36c204d5'
convert(100, 'USD', 'XAF', api)

Can't connect to internet... 
Using old rate from Mon, 30 Aug 2021 00:02:31 UTC


55638.0

## The future modifications of this function intends to implement error handling